In [29]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs']

In [30]:
def get64Samples(real, imag):

    # Number of signalsd
    num_signals = real.shape[0]  # 3 in this case
    
    # Initialize lists to store the focused samples
    imp_real_parts = []
    imp_imag_parts = []
    img_mag_parts = []
    
    for i in range(num_signals):
        # Calculate the magnitude
        magnitude = np.abs(real[i] + 1j * imag[i])
        
        # find the peak index
        peak_index = np.argmax(magnitude)
        
        # Calculate the start and end indices for the focused part
        start_index = max(0, peak_index - 32)
        end_index = min(magnitude.shape[0], peak_index + 32)
        
        # Extract the part of the signal around the peak
        real_part_focus = real[i, start_index:end_index]
        imag_part_focus = imag[i, start_index:end_index]
        mag_part_focus = magnitude[start_index:end_index]
        
        imp_real_parts.append(real_part_focus)
        imp_imag_parts.append(imag_part_focus)
        img_mag_parts.append(mag_part_focus)
        

    # Convert lists back to arrays for further processing if needed
    imp_real_parts = np.array(imp_real_parts)
    imp_imag_parts = np.array(imp_imag_parts)
    img_mag_parts = np.array(img_mag_parts)

    return imp_real_parts, imp_imag_parts, img_mag_parts

In [31]:
# Define channels
alice_channel = 3  
eve_channel = 6  

# Extract data for ALICE and BOB channels
alice_CIRs = data_cir[:, alice_channel, :, :]  # Shape: (9797, 251, 2)
eve_CIRs = data_cir[:, eve_channel, :, :]  # Shape: (9797, 251, 2)

alice_real_251 = alice_CIRs[:, :, 0]
alice_imag_251 = alice_CIRs[:, :, 1]
alice_real, alice_imag, alice_mag = get64Samples(alice_real_251, alice_imag_251)
alice_features = np.hstack((alice_real, alice_imag, alice_mag))

eve_real_251 = eve_CIRs[:, :, 0]
eve_imag_251 = eve_CIRs[:, :, 1]
eve_real, eve_imag, eve_mag = get64Samples(eve_real_251, eve_imag_251)
eve_features = np.hstack((eve_real, eve_imag, eve_mag))

# Create labels for Alice and Eve.
alice_labels = np.zeros(alice_features.shape[0])  # Label '0' for Alice. (9797,)
eve_labels = np.ones(alice_features.shape[0])     # Label '1' for Eve. (9797,)

# # Combine data and labels.
atoms = np.vstack((alice_features, eve_features))  # (19594, 192)
true_labels = np.hstack((alice_labels, eve_labels))  # (19594,)

In [32]:
# Step 3: Split the Data into Training and Test Sets
trainData, testData, trainLabel, testLabel = train_test_split(atoms, true_labels, test_size=0.2, random_state=42, stratify=true_labels)

# Step 4: Form the Dictionary D
D = trainData.T
print(D.shape)


(192, 15675)


In [33]:
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=20):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

In [34]:
# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual


In [35]:
# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    coefficients = find_sparse_coefficients(tSample, D)
    # print(coefficients.shape)
    residuals = []

    # Calculate residual for each class
    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        print(class_indices.shape)
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class


In [36]:
# Step 8: Classifying Test Data and Evaluating the Model

# testData (40, 192)
predictions = []
for testSample in testData[:1]:
    # print(testSample.shape)
    predicted_class = classify_signal(testSample, D, trainLabel)
    predictions.append(predicted_class)

# Convert predictions to a numpy array for comparison
predictions = np.array(predictions)
# print(predictions)

# Step 9: Calculate Accuracy
accuracy = np.mean(predictions == testLabel)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

(7837,)
(7838,)
Classification Accuracy: 50.01%


In [37]:
# Calculate confusion matrix
print(f"\nTotal testing channel: {testData.shape}")

tn, fp, fn, tp = confusion_matrix(testLabel, predictions, labels=[0, 1]).ravel()
# new comment
print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# # Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# # False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# # Gamma calculation
gamma = (tp + fn) / (tn + fp)

# # Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Total testing channel: (3919, 192)


ValueError: Found input variables with inconsistent numbers of samples: [3919, 1]